In [ ]:
# IVIM Reduction

In [1]:
# linear algorithm
from itertools import chain, repeat, count, islice
from collections import Counter
import numpy as np
import datetime
import os

class linear_reduction:
    def __init__(self, data, bvals, opt_bvals, dirName):
        """
        data - 100x100xn matrix
        bvals - 1xn array 
        """
        
        self.data = data
        self.bvals = bvals
        self.opt_bvals = opt_bvals
    
    def create_outputfolder(dirName):
        try:
            # Create target Directory
            os.mkdir(dirName)
            print("Directory " , dirName ,  " Created ") 
        except FileExistsError:
            print("Directory " , dirName ,  " already exists")
            
    def subsampling(self, r, bvals):
        """
        
        """
        def combinations_without_repetition(r, iterable=None, values=None, counts=None):
            """ Generate combinations
            """
            if iterable:
                values, counts = zip(*Counter(iterable).items())
            f = lambda i, c: chain.from_iterable(map(repeat, i, c))
            n = len(counts)
            indices = list(islice(f(count(), counts), r))
            if len(indices) < r:
                return
            while True:
                yield list(values[i] for i in indices)
                for i, j in zip(reversed(range(r)), f(reversed(range(n)), reversed(counts))):
                    if indices[i] != j:
                        break
                else:
                    return
                j = indices[i]+1
                for i, j in zip(range(i, r), f(count(j), counts[j:])):
                    indices[i] = j
        
        # Generate subsampling
        subsampling = list(combinations_without_repetition(r, iterable=bvals))

        return subsampling

    def data_sub(self, data, bvals, sub_bvals):
        def data_bval_selection(data, bvals, sub_bvals):
            data_new = np.zeros((len(sub_bvals)), dtype=object)
            bvals = bvals.tolist()
            for i in range(len(sub_bvals)):
                data_new[i] = bvals.index(sub_bvals[i])
            
            return data_new
        
        data_new = np.zeros(data.shape)
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                data_new[i,j] = data_bval_selection(data, bvals, sub_bvals)
                
        return data_new
    
    
    
    def fit(self, func, data, bvals):
        """
        func -> [ d, d_star, f] = func(signal,bvals)
        """
        output = []
        for i in range(100):
            for j in range(100):
                d, d_star, f = 0, 0, 0
                d, d_star, f = func(data[i,j],bvals)
                output.append([ d, d_star, f])
        
        return output
    
    def rrmse(self, output,real):
    
        d, pd, f = real
        d_sub, pd_sub, f_sub = output

        rrmseD= np.zeros(d_sub.shape[-1])
        rrmsepD= np.zeros(d_sub.shape[-1])
        rrmsef= np.zeros(d_sub.shape[-1])

        rrmseDsum, rrmsepDsum, rrmsefsum = [0, 0, 0]

        for i in range(d_sub.shape[0]):
            rrmseDsum += (d-d_sub[i])**2
            rrmsepDsum += (pd-pd_sub[i])**2
            rrmsefsum += (f-f_sub[i])**2

        rrmseD = np.sqrt(rrmseDsum / d_sub.shape[0])/np.mean(d_sub)
        rrmsepD = np.sqrt(rrmsepDsum / pd_sub.shape[0])/np.mean(pd_sub)
        rrmsef = np.sqrt(rrmsefsum / f_sub.shape[0])/np.mean(f_sub)

        return [rrmseD,rrmsepD,rrmsef]

data = []
bvals = []
opt_bvals = []
x = datetime.datetime.now()
dirName = 'output-' + x.strftime("%c")
lin = linear_reduction(data, bvals, opt_bvals, dirName)
    

NameError: name 'dirName' is not defined